In [ ]:
# data set link- https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset

In [1]:
import os
import cv2
import numpy as np
import pickle
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from pennylane import numpy as pnp
from math import ceil, log2

# === Setup ===
dataset_path = r"D:\Images_anantwave\Mulit_class\Plant_disease\New Plant Diseases Dataset(Augmented)\train"
categories = [folder for folder in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, folder))]
num_images_per_category = 1000  # You can adjust this if needed

# === Feature Extraction using SIFT ===
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if descriptors is None:
        return np.zeros(128)
    return descriptors.mean(axis=0)

# === Load Dataset in Parallel ===
def process_image(category, image_file):
    image_path = os.path.join(dataset_path, category, image_file)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is not None:
        feat = extract_features(image)
        return feat, category
    return None, None

# === Feature Extraction Loop ===
all_features, labels = [], []
for category in categories:
    print(f"Processing {category} images...")
    path = os.path.join(dataset_path, category)
    valid_exts = [".jpg", ".jpeg", ".png"]
    image_files = [f for f in os.listdir(path) if os.path.splitext(f)[1].lower() in valid_exts][:num_images_per_category]

    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda f: process_image(category, f), image_files), total=len(image_files)))
    
    for feat, label in results:
        if feat is not None:
            all_features.append(feat)
            labels.append(label)

# === Standardization + PCA ===
scaler = StandardScaler()
scaled = scaler.fit_transform(all_features)

pca = PCA(0.95)  # retain 95% variance
reduced = pca.fit_transform(scaled)

# === Quantum Configuration ===
original_dim = reduced.shape[1]
n_qubits = int(ceil(log2(original_dim)))
dim = 2**n_qubits

# Pad to match 2^n
def pad(vec, target_dim):
    if len(vec) < target_dim:
        return np.pad(vec, (0, target_dim - len(vec)))
    else:
        return vec[:target_dim]

# Quantum Device
dev = qml.device("lightning.qubit", wires=n_qubits)

# QAOA Layer
def qaoa_layer(gamma, beta, wires):
    for i in range(len(wires)):
        qml.CZ(wires=[wires[i], wires[(i+1)%len(wires)]])
    for i in range(len(wires)):
        qml.RX(gamma, wires=i)
        qml.RZ(beta, wires=i)

# QNode
@qml.qnode(dev)
def quantum_model(x, gamma=0.5, beta=0.5):
    qml.AmplitudeEmbedding(x / np.linalg.norm(x), wires=range(n_qubits), normalize=True, pad_with=0.0)
    qaoa_layer(gamma, beta, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# === Encode All with Quantum Circuit ===
quantum_outputs = []
print("Quantum encoding in progress...")
for vec in tqdm(reduced):
    vec_padded = pad(vec, dim)
    vec_pnp = pnp.array(vec_padded, requires_grad=False)
    result = quantum_model(vec_pnp)
    quantum_outputs.append(result)

# === Save Output ===
final_data = {
    "quantum_encoded": quantum_outputs,
    "labels": labels
}
with open("quantum_plant_disease_data.pkl", "wb") as f:
    pickle.dump(final_data, f)

print("Quantum encoding complete for Plant Disease Dataset (38 classes)!")


Processing Apple___Apple_scab images...


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 156.60it/s]


Processing Apple___Black_rot images...


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 105.54it/s]


Processing Apple___Cedar_apple_rust images...


...

Processing Tomato___Tomato_Yellow_Leaf_Curl_Virus images...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:13<00:00, 71.98it/s]


Quantum encoding in progress...


100%|███████████████████████████████████████████████████████████████████████████| 38000/38000 [02:26<00:00, 258.90it/s]


Quantum encoding complete for Plant Disease Dataset (38 classes)!


In [5]:
import pickle

# Load the quantum encoded data
with open("quantum_plant_disease_data.pkl", "rb") as f:
    data = pickle.load(f)

# Print the first 5 quantum encoded vectors and their labels
print(" Quantum Encoded Outputs (First 5):")
for i in range(5):
    print(f"Sample {i+1}:")
    print("  Label  :", data["labels"][i])
    print("  Encoding:", data["quantum_encoded"][i])
    print()

 Quantum Encoded Outputs (First 5):
Sample 1:
  Label  : Apple___Apple_scab
  Encoding: [array(0.81434411), array(0.64074246), array(0.50451672), array(-0.00244807), array(0.2390138)]

Sample 2:
  Label  : Apple___Apple_scab
  Encoding: [array(0.77922596), array(0.62836426), array(0.28125055), array(-0.09541319), array(0.10809613)]

Sample 3:
  Label  : Apple___Apple_scab
  Encoding: [array(0.75372835), array(0.61291867), array(0.42340203), array(0.18105615), array(-0.07074394)]

Sample 4:
  Label  : Apple___Apple_scab
  Encoding: [array(0.79523267), array(0.73095016), array(0.38394365), array(0.02379645), array(0.05255881)]

Sample 5:
  Label  : Apple___Apple_scab
  Encoding: [array(0.7430567), array(0.53996844), array(0.40175522), array(0.22602823), array(0.07929385)]

